In [ ]:
%cd C:\Users\lance\Documents\20-24-UT-Files\Competitions\hacktx-2023\scratch

In [25]:
# clone YOLOv5 repository
#!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
#!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

C:\Users\lance\Documents\20-24-UT-Files\Competitions\hacktx-2023\scratch\yolov5


## Download Dataset

In [26]:
# install dependencies as necessary
#!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.1.0+cpu CPU


In [27]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="fTR9oznM3hehEtAWXLzL")
project = rf.workspace("alex-hyams-cosqx").project("cash-counter")
dataset = project.version(11).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Cash-Counter-11 in yolov5pytorch::  96%|█████████▌| 5862/6118 [00:02<00:00, 2877.92it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Cash-Counter-11\\valid\\images\\american-coins-quarters-pennies-laying-wooden-table-cape-may-county-new-jersey-nj-spring-american-coins-laying-wooden-118503082_jpg.rf.a877ced693fb74d3ca0e08e12052ac16.jpg'

In [30]:
%cat {dataset.location}/data.yaml

UsageError: Line magic function `%` not found.


##Train Model

In [8]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

print(num_classes)

NameError: name 'dataset' is not defined

In [6]:
#this is the model configuration we will use for our tutorial
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [7]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [8]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [16]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
2023-10-21 21:43:44.244114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-21 21:43:45.156282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=./models/custom_yolov5s.yaml, data=/content/yolov5/Cash-Counter-11/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_per

In [13]:
%cd ..
!zip -r yolov5 yolov5

Streaming output truncated to the last 5000 lines.
  adding: yolov5/Cash-Counter-11/train/images/IMG_2055_jpg.rf.44681ec0ecb19151ff3f71794affb353.jpg (deflated 0%)
  adding: yolov5/Cash-Counter-11/train/images/17E93EB2-F72E-40F3-8679-171BDD342C06.rf.8d25ea81ee3285eeca689a81b312c9e3.jpg (deflated 68%)
  adding: yolov5/Cash-Counter-11/train/images/IMG_2105_jpg.rf.455e05dd0edebe6f516cbc86b53da9f8.jpg (deflated 1%)
  adding: yolov5/Cash-Counter-11/train/images/001__1-Cent_usa_jpg.rf.480b158fc001ebf7cca45623645eff83.jpg (deflated 0%)
  adding: yolov5/Cash-Counter-11/train/images/IMG_2160_jpeg_jpg.rf.eae3264c0ebadb5f497b66e6c9792325.jpg (deflated 0%)
  adding: yolov5/Cash-Counter-11/train/images/C_White_Clean_jpg.rf.c570e3e292a15ab0bd7da3813c9aa670.jpg (deflated 2%)
  adding: yolov5/Cash-Counter-11/train/images/IMG_8483_jpg.rf.f3f6d6bd7435e67c0b368bf0ce644ba7.jpg (deflated 1%)
  adding: yolov5/Cash-Counter-11/train/images/IMG_2044_jpg.rf.f2fef14c2854debdccf3d24d1c6bbc38.jpg (deflated 0%)
  a